In [2]:
import os
import sys
import pandas as pd

In [3]:
create_forecast_basic_software_location = os.path.dirname(os.getcwd())

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(create_forecast_basic_software_location))

software_folder_location=df_inputs_outputs['location'][0]

sys.path.append(create_forecast_basic_software_location)

TAZ_V4_date=df_inputs_outputs['location'][1]

In [4]:
from global_functions import up_load_shp, make_point, add_geo_info_shp

In [5]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [6]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}.shp'.format(software_folder_location, TAZ_V4_date))

In [8]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'jeru_metro_jtmt_border_240402', 'jeru_metro')

In [30]:
taz.to_file(r"{}\background_files\TAZ_V4_{}_with_geo_info.shp".format(software_folder_location, file_date), encoding='utf-8')